In [1]:
import cv2
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input
import tensorflow.keras.backend as K
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetB0
from keras.applications import ResNet50


In [3]:
# Charger les données
data_train = pd.read_csv('./eyes-dataset/Training_Set/Training_Set/RFMiD_Training_Labels.csv')
data_test = pd.read_csv('./eyes-dataset/Test_Set/Test_Set/RFMiD_Testing_Labels.csv')
data_evaluation = pd.read_csv('./eyes-dataset/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv')

# Définir le chemin du dossier contenant les images
image_folder_train = './eyes-dataset/Training_Set/Training_Set/Training/'  # Mettez le chemin approprié
image_folder_eval = './eyes-dataset/Evaluation_Set/Evaluation_Set/Validation/'  #
image_folder_test= './eyes-dataset/Test_Set/Test_Set/Test/'





In [4]:
#ajout de la colonne image path a chaque datset 

data_train['path'] = [f'./eyes-dataset/Training_Set/Training_Set/Training/{id}' for id in data_train['ID']]
data_test['path'] = [f'./eyes-dataset//Test_Set/Test_Set/Test/{id}' for id in data_test['ID']]
data_evaluation['path'] = [f'./eyes-dataset/Evaluation_Set/Evaluation_Set/Validation/{id}' for id in data_evaluation['ID']]

In [5]:
#Après l'ajout du path dans une olonne on peut donc drop l'id il ne sert plus à rien 

data_train.drop(columns=['ID'], inplace=True)
data_test.drop(columns=['ID'], inplace=True)
data_evaluation.drop(columns=['ID'], inplace= True)

In [9]:
cols_to_exclude = ['path'] 

# Suppression des colonnes non pertinentes
data_for_labels = data_train.drop(columns=cols_to_exclude)

# Obtention de la liste des maladies (colonnes restantes) pour l'entraînement, la validation, et les ensembles de tests
y_train = y_val = y_test = list(data_for_labels.columns)

# Détermination du nombre de maladies uniques
unique_diseases = len(y_train)

In [19]:
print(y_train)

['Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'AH', 'ODP', 'ODE', 'AION', 'RS', 'CRS', 'EDN', 'RPEC']


In [10]:
data_evaluation.head()

,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,LS,MS,...,CRVO,AH,ODP,ODE,AION,RS,CRS,EDN,RPEC,img_path
0,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,./eyes-dataset/Evaluation_Set/Evaluation_Set/V...
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,./eyes-dataset/Evaluation_Set/Evaluation_Set/V...
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,./eyes-dataset/Evaluation_Set/Evaluation_Set/V...
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,./eyes-dataset/Evaluation_Set/Evaluation_Set/V...
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,./eyes-dataset/Evaluation_Set/Evaluation_Set/V...


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Création d'une instance de ImageDataGenerator (vous pouvez éventuellement normaliser vos données ici)
datagen = ImageDataGenerator(rescale=1./255.)  # normalisation des pixels

# Supposons que vous avez des dataframes distincts pour l'entraînement, le test et la validation :
# data_train, data_val, data_test et que 'img_path' est la colonne avec le chemin des images.

# Chargement des données d'entraînement
train_data = datagen.flow_from_dataframe(
    dataframe=data_train,
    directory=None,  
    x_col='path',  
    y_col=y_train,  
    target_size=(356, 536),  
    color_mode='rgb',  
    class_mode='raw',  
    batch_size=32,  
    shuffle=True  
)

# Chargement des données de validation
val_data = datagen.flow_from_dataframe(
    dataframe=data_evaluation,  
    directory=None,
    x_col='img_path',
    y_col=y_val,
    target_size=(356, 536),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,  
    shuffle=False  
)

# Chargement des données de test
test_data = datagen.flow_from_dataframe(
    dataframe=data_test,  
    directory=None,
    x_col='img_path',
    y_col=y_test,
    target_size=(356, 536),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,  
    shuffle=False
)




Found 1920 validated image filenames.
Found 640 validated image filenames.
Found 640 validated image filenames.


In [12]:
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf

def augment_image(image):
    
    image = preprocessing.RandomFlip("horizontal")(image)
    
    image = preprocessing.RandomFlip("vertical")(image)
 
    return image

# Création d'un modèle séquentiel pour la couche d'augmentation de données
data_augmentation = keras.Sequential([
    keras.layers.Lambda(augment_image)  
], name="data_augmentation")




In [13]:
pre_model = ResNet50(weights='imagenet', input_shape=(356,536,3), include_top=False)
pre_model.trainable = False

In [14]:



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import MobileNetV2

# Define the input shape
input_shape = (356, 536, 3)  

base_model = MobileNetV2(input_shape=input_shape,
                         include_top=False,  
                         weights='imagenet')  

base_model.trainable = False  

# Create a new sequential model and add the layers, including the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(unique_diseases, activation='sigmoid')  
])

# Compile the model
recall = tf.keras.metrics.Recall(name="recall", thresholds=0.1)
acc = tf.keras.metrics.BinaryAccuracy(name="binary_acc", threshold=0.1)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
              loss='binary_crossentropy', 
              metrics=[acc, recall])

# Display the model summary
model.summary()


model.save('my_sequential_model.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 12, 17, 1280)      2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 21)                1

In [16]:

# Importations nécessaires
from tensorflow.keras.callbacks import EarlyStopping, Callback






# Lancement de l'entraînement du modèle.
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
   
    verbose=1  
)

model.save('my_model_ml.h5')  



Epoch 1/10
60/60 [==============================] - 269s 5s/step - loss: 0.1762 - binary_acc: 0.8574 - recall: 0.7479 - val_loss: 0.1602 - val_binary_acc: 0.8027 - val_recall: 0.8575
Epoch 2/10
60/60 [==============================] - 270s 5s/step - loss: 0.1472 - binary_acc: 0.8842 - recall: 0.8030 - val_loss: 0.1374 - val_binary_acc: 0.8693 - val_recall: 0.8373
Epoch 3/10
60/60 [==============================] - 286s 5s/step - loss: 0.1352 - binary_acc: 0.8979 - recall: 0.8314 - val_loss: 0.1426 - val_binary_acc: 0.8513 - val_recall: 0.8391
Epoch 4/10
60/60 [==============================] - 272s 5s/step - loss: 0.1280 - binary_acc: 0.8979 - recall: 0.8443 - val_loss: 0.1329 - val_binary_acc: 0.8958 - val_recall: 0.8144
Epoch 5/10
60/60 [==============================] - 276s 5s/step - loss: 0.1197 - binary_acc: 0.9049 - recall: 0.8589 - val_loss: 0.1239 - val_binary_acc: 0.9003 - val_recall: 0.8373
Epoch 6/10
60/60 [==============================] - 281s 5s/step - loss: 0.1196 - bin

In [17]:
model.evaluate(test_data)

20/20 [==============================] - 79s 4s/step - loss: 0.1324 - binary_acc: 0.8940 - recall: 0.8285


[0.1324252486228943, 0.8940476179122925, 0.8284960389137268]